# Building A Chatbot

here we will go over an example of how to design and implement an LLM-powered chatbot .This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we will build will only use the language model to have conversation.There are several other related concepts we may be looking for :

  . Conversational RAG : Enable a ChatBot experience     over an external source of data
  . Agents: Build a chatbot that can take actions

here we will cover basics which will be helpful for these two more advanced topics


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
## calling our llm model
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)


In [ ]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content='hi,my name is kafil and i am llm engineer')])

AIMessage(content="Hi Kafil, it's nice to meet you! \n\nThat's fascinating! As an AI, I'm always interested in learning more about the people who build and work with models like me. What kind of work are you doing as an LLM engineer? \n\nAre you working on a specific project you'd like to talk about, or are you just interested in chatting about the field?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 22, 'total_tokens': 109, 'completion_time': 0.158181818, 'prompt_time': 0.001324689, 'queue_time': 0.251691882, 'total_time': 0.159506507}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--49490f97-b2c7-41e4-8594-1e49adfaf1e6-0', usage_metadata={'input_tokens': 22, 'output_tokens': 87, 'total_tokens': 109})

In [ ]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content='hi,my name is kafil and i am llm engineer'),
        AIMessage(content="Hi Kafil, it's nice to meet you! \n\nThat's fascinating! As an AI, I'm always interested in learning more about the people who build and work with models like me. What kind of work are you doing as an LLM engineer? \n\nAre you working on a specific project you'd like to talk about, or are you just interested in chatting about the field?\n"),
        HumanMessage(content='hey whats my name and what do i do?')
    ]
)
## we observe that it is able to rememmber our history of conversation

AIMessage(content="You said your name is Kafil and you're an LLM engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work? 😊  I'm eager to learn more. \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 127, 'total_tokens': 176, 'completion_time': 0.089090909, 'prompt_time': 0.003368338, 'queue_time': 0.253719919, 'total_time': 0.092459247}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--e7cd722f-a618-4932-ba7c-f5fce7012606-0', usage_metadata={'input_tokens': 127, 'output_tokens': 49, 'total_tokens': 176})

# sessions are managed --> message history

we can use a Message History class to wrap our model and make it stateful. This will help keep track of inputs and outputs of the model , and store them in some Datastore.
Future Interactions will then Load Those Messages and pass into the Chain AS Part of the Input.

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store={}

## when different users are chatting with llm model how are we gonna make sure that 1 session is ci=ompletely diffeerent from another session.
## for that we will be creating a  very important function named get_session_history
def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()

    return store[session_id]


# this fn will be creating a session id of str type and return type will be (-> basechatmessagehistory)
# session id will distinguish 1 session id with another

with_message_history=RunnableWithMessageHistory(model,get_session_history)


In [6]:
config={"configurable":{"session_id":"chat1"}}

In [ ]:
response=with_message_history.invoke(
    [
        HumanMessage(content='hi i am kafil . i m an ai engineer')
    ],config=config
)

In [11]:
response.content

"Hi Kafil!  Welcome!\n\nIt's great to meet another AI enthusiast.  What kind of AI work are you involved in? 🤖  I'm always eager to learn about new projects and applications. 😊 \n\n"

In [12]:
with_message_history.invoke(
    [HumanMessage(content="whats my name?")],config=config
)

AIMessage(content='Your name is Kafil.  😊  \n\nI remembered from our earlier conversation!  What can I help you with today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 159, 'total_tokens': 188, 'completion_time': 0.052727273, 'prompt_time': 0.004531875, 'queue_time': 0.256362352, 'total_time': 0.057259148}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f0867ed9-2f24-4e94-b02b-94e00afdc9d6-0', usage_metadata={'input_tokens': 159, 'output_tokens': 29, 'total_tokens': 188})

In [13]:
## changing config i.e session id
config1={"configurable":{"session_id":"chat2"}}
with_message_history.invoke(
    [HumanMessage(content="whats my name?")],config=config1
)

AIMessage(content="As an AI, I have no access to your personal information, including your name. If you'd like to tell me your name, I'd be happy to know! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 13, 'total_tokens': 53, 'completion_time': 0.072727273, 'prompt_time': 0.001249271, 'queue_time': 0.249029944, 'total_time': 0.073976544}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b2083744-ec20-41dc-b61b-220b31c64ca3-0', usage_metadata={'input_tokens': 13, 'output_tokens': 40, 'total_tokens': 53})

In [14]:
with_message_history.invoke(
    [HumanMessage(content="my name is kaif")],config=config1
)

AIMessage(content="Hi Kaif, it's nice to meet you!  \n\nIs there anything I can help you with today?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 66, 'total_tokens': 95, 'completion_time': 0.052727273, 'prompt_time': 0.002196421, 'queue_time': 0.254026794, 'total_time': 0.054923694}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--679f4b4f-c9ad-468c-a360-279c7ab85fc0-0', usage_metadata={'input_tokens': 66, 'output_tokens': 29, 'total_tokens': 95})

In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="whats my name?")],config=config1
)
response.content

'Your name is Kaif!  \n\nI remember. 😊  \n\nIs there something else I can help you with?'

## Prompt Templates

Prompt Templates help to turn raw user information into a formt that the LLM can work with . In this case , the raw user input is just a message, which we are passing to llm . Lets make it a bit more complicated.
First, lets add in a system message with some custom instruction (but still taking m,essage as input ),next, we will add more input besides just the message

In [17]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages([
    ("system","you are a helpful assistant . answer all questions to the best of your ability"),
    MessagesPlaceholder(variable_name="messages")
])

chain=prompt|model

In [18]:
chain.invoke({"messages":[HumanMessage(content="hi m,y name is kafil")]})

AIMessage(content="Hello Kafil! 👋\n\nIt's nice to meet you. I'm happy to help with any questions you have. Just ask away! 😊  \n\nWhat can I do for you today?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 32, 'total_tokens': 78, 'completion_time': 0.083636364, 'prompt_time': 0.001555529, 'queue_time': 0.244384456, 'total_time': 0.085191893}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a67f3396-8125-460a-881b-ab7d93c341bb-0', usage_metadata={'input_tokens': 32, 'output_tokens': 46, 'total_tokens': 78})

In [19]:
# how to invoke this with chat message history
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [21]:
config2={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="my name is kafil")],config=config2
)
response.content

"Hi Kafil, it's nice to officially meet you!  \n\nIs there anything I can help you with today?  😊 \n\n"

In [22]:
## Adding more complexity to my prompt
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","you are a helpful assistant . answer all questions to the best of your ability in this {language}"),
        MessagesPlaceholder(variable_name="messages")
])

chain=prompt|model

In [24]:
## whenever i need to invoke this chain there are multiple parameters which i need to give
response=chain.invoke({"messages":[HumanMessage(content="hi my name is kafil aslam")],"language":"hindi"})
response.content

'नमस्ते काफ़िल असलम!  \n\nआप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी कोशिश करूंगा आपको मदद करने के लिए। 😊 \n\nआपका सवाल क्या है?  \n'

lets now wrap  this complicated chain in message history class . this time , bcz there are multiple keys as input , we need to specify correct key to use to save gthe chat history

In [25]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
    
)

In [35]:
config4={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {'messages':[HumanMessage(content='hi i am kafil')],"language":"English"},
    config=config4
)
response.content

'Hi Kafil! 👋 Nice to meet you. What can I do for you today?  \n'

In [39]:
response=with_message_history.invoke(
    {'messages':[HumanMessage(content='whats my name')],"language":"English"},
    config=config4
)
response.content

"Your name is Kafil.  \n\nIs there anything else you'd like to know or talk about?  I'm here to help! 😊 \n"

## Managing the Conversation History

one important concept to understand when building chatbots is how to manage conversation history. if left unmanaged , the list of messages will grow unbounded  and potentially overflow the context window of the LLM. Therefore , it is important to add a step that limits the size of messages that we are passing in . 

"trim_messages" helper is to reduce how many messages we're sending to the model . The Trimmer allows us to specify how many tokens we want to keep , along with other parameters like if we wnt to always keep the system message and weather to allow partial messages

In [42]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer=trim_messages(
    max_tokens=40,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"



)

messages=[
    SystemMessage(content="you r a good assistant"),
    HumanMessage(content="hi , i am bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="i like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="having fun"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)




[SystemMessage(content='you r a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [45]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)

response=chain.invoke({
    "messages":messages + [HumanMessage(content="what ice cream do i like ?")],
    "language":"english"
}
)
response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences. \n\nWhat's your favorite flavor? 😊  🍦\n"

In [46]:
response=chain.invoke({
    "messages":messages + [HumanMessage(content="what math problem did i asked")],
    "language":"english"
}
)
response.content

'You asked: " whats 2 + 2" \n\n\nLet me know if you want to try another one! 😊  \n'

In [47]:
## lets wrap inside message history
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config5={"configurable":{"session_id":"chat5"}}



In [ ]:
response=with_message_history.invoke(
    {
        "messages":messages + [HumanMessage(content="whats my name")],
        "language":"english"
    },
    config=config5
)
response.content

"As an AI, I have no memory of past conversations and don't know your name.  What's your name? 😊  \n"